In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip install transformers
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# Freeze the first two BERT encoder layers
for param in model.bert.encoder.layer[:2].parameters():
    param.requires_grad = False


In [ ]:
from torch import nn

# Replace the classifier head
num_labels = 2
model.classifier = nn.Linear(768, num_labels)


In [ ]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding

# Load Yelp dataset
dataset = load_dataset('yelp_polarity')#, split='train')

# Tokenize Yelp dataset
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size=16)

# Create data collator with padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2375 [00:00<?, ?ba/s]

In [ ]:
from datasets import load_metric
import numpy as np


# Load the accuracy metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
tokenized_dataset["train"][4]

{'text': 'All the food is great here. But the best thing they have is their wings. Their wings are simply fantastic!!  The \\""Wet Cajun\\"" are by the best & most popular.  I also like the seasoned salt wings.  Wing Night is Monday & Wednesday night, $0.75 whole wings!\\n\\nThe dining area is nice. Very family friendly! The bar is very nice is well.  This place is truly a Yinzer\'s dream!!  \\""Pittsburgh Dad\\"" would love this place n\'at!!',
 'label': 1,
 'input_ids': [101,
  2035,
  1996,
  2833,
  2003,
  2307,
  2182,
  1012,
  2021,
  1996,
  2190,
  2518,
  2027,
  2031,
  2003,
  2037,
  4777,
  1012,
  2037,
  4777,
  2024,
  3432,
  10392,
  999,
  999,
  1996,
  1032,
  1000,
  1000,
  4954,
  6187,
  19792,
  1032,
  1000,
  1000,
  2024,
  2011,
  1996,
  2190,
  1004,
  2087,
  2759,
  1012,
  1045,
  2036,
  2066,
  1996,
  28223,
  5474,
  4777,
  1012,
  3358,
  2305,
  2003,
  6928,
  1004,
  9317,
  2305,
  1010,
  1002,
  1014,
  1012,
  4293,
  2878,
  4777,
  99

In [ ]:
train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(5000))
eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(5000))

In [ ]:
from transformers import Trainer, TrainingArguments

# Set training arguments
output_dir = 'output/directory'
num_train_epochs = 5
per_device_train_batch_size = 16
per_device_eval_batch_size=64
gradient_accumulation_steps = 1
learning_rate = 5e-5
warmup_steps = 500
logging_dir = 'log/directory'
eval_strategy = "epoch"
weight_decay=0.01
logging_steps=500               # log training loss every 500 steps
eval_steps=1000

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    evaluation_strategy=eval_strategy,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    weight_decay=weight_decay,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    warmup_steps=warmup_steps,
    logging_dir=logging_dir,
    logging_steps=logging_steps,
    eval_steps=eval_steps,
    push_to_hub=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Fine-tune the model
trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1565
  Number of trainable parameters = 95308034


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.436515,0.940000
2,0.024700,0.326246,0.934400
3,0.024700,0.294735,0.942000
4,0.049200,0.334686,0.947600
5,0.009300,0.360029,0.948000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 64
Saving model checkpoint to output/directory/checkpoint-500
Configuration saved in output/directory/checkpoint-500/config.json
Model weights saved in output/directory/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignore

TrainOutput(global_step=1565, training_loss=0.02660895227052914, metrics={'train_runtime': 769.2053, 'train_samples_per_second': 32.501, 'train_steps_per_second': 2.035, 'total_flos': 6577776384000000.0, 'train_loss': 0.02660895227052914, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 64


{'eval_loss': 0.3600292503833771,
 'eval_accuracy': 0.948,
 'eval_runtime': 37.0984,
 'eval_samples_per_second': 134.777,
 'eval_steps_per_second': 2.129,
 'epoch': 5.0}